In [8]:
!pip uninstall --yes nvidia-ml-py3
!pip uninstall --yes pandas-profiling
!pip uninstall --yes scipy
!pip install scipy==1.7.2
!pip install pandas-profiling
!pip install auto-sklearn==0.14.0

Found existing installation: nvidia-ml-py3 7.352.0
Uninstalling nvidia-ml-py3-7.352.0:
  Successfully uninstalled nvidia-ml-py3-7.352.0
Found existing installation: pandas-profiling 1.4.1
Uninstalling pandas-profiling-1.4.1:
  Successfully uninstalled pandas-profiling-1.4.1
Found existing installation: scipy 1.4.1
Uninstalling scipy-1.4.1:
  Successfully uninstalled scipy-1.4.1
     |████████████████████████████████| 38.2 MB 25 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 1.0.61 requires nvidia-ml-py3, which is not installed.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 261 kB 5.1 MB/s 
     |████████████████████████████████| 62 kB 890 kB/s 
     |████████████████████████████████| 675 kB 45.0 MB/s 
     |████████████████████████████████| 303 kB 55.2 

In [3]:
import numpy as np
import pandas as pd

In [1]:
# https://drive.google.com/file/d/1GYV0XAVBgO95Lf-lTuMBf6YD2KaIjj8E/view?usp=sharing
# https://drive.google.com/file/d/1eJRZdF1yf7a-CjAJUwTno6IaYzlKQChM/view?usp=sharing
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1GYV0XAVBgO95Lf-lTuMBf6YD2KaIjj8E' -O test.csv
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1eJRZdF1yf7a-CjAJUwTno6IaYzlKQChM' -O train.csv


--2021-11-11 12:20:04--  https://docs.google.com/uc?export=download&id=1GYV0XAVBgO95Lf-lTuMBf6YD2KaIjj8E
Resolving docs.google.com (docs.google.com)... 74.125.26.139, 74.125.26.101, 74.125.26.138, ...
Connecting to docs.google.com (docs.google.com)|74.125.26.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-8o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/js2ks9jeq59obieh7ql6vhljfotfn8ci/1636633200000/18132256971849676264/*/1GYV0XAVBgO95Lf-lTuMBf6YD2KaIjj8E?e=download [following]
--2021-11-11 12:20:06--  https://doc-0s-8o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/js2ks9jeq59obieh7ql6vhljfotfn8ci/1636633200000/18132256971849676264/*/1GYV0XAVBgO95Lf-lTuMBf6YD2KaIjj8E?e=download
Resolving doc-0s-8o-docs.googleusercontent.com (doc-0s-8o-docs.googleusercontent.com)... 172.217.193.132, 2607:f8b0:400c:c03::84
Connecting to doc-0s-8o-docs.googleusercontent.com (doc-0s-8o-docs

In [4]:
train_numData = pd.read_csv('train.csv')

In [ ]:
def convertCategoricalsToNumerics(df):
    
    categorical_fields = df.select_dtypes(include=['object'])
    categorical_fields = categorical_fields.drop('class',axis=1)
    
    for col in df.columns:
        if col in categorical_fields.columns:
                       
            # Counting the different categories in the column
            countOrdered_cats = list(dict(categorical_fields[col].value_counts()).keys()) # descending order
            
            
            # Creating a list of numeric replacements
            num_replacements = list(np.arange(len(countOrdered_cats))+1)
            
            # Dictionary of replacements to pass to df.replace()
            replacements = dict(zip(countOrdered_cats, num_replacements))
    
            cd = df[col].replace(replacements, inplace=True)
        
        else:
            pass
        
    return df
    
train_numData = convertCategoricalsToNumerics(train_data)
train_numData

In [5]:
X = train_numData.loc[:, train_numData.columns != 'class']

Y = train_numData.loc[:, train_numData.columns == 'class']

In [6]:
import autosklearn.classification
import autosklearn.metrics

from sklearn.model_selection import train_test_split

test_size = 0.2
shuffle = True

train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=test_size, shuffle=shuffle)

In [8]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV

import ConfigSpace.hyperparameters as CSH

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier


numeric_transformer = make_pipeline(StandardScaler())

Preprocessor = ColumnTransformer(
 transformers=[
 ('numeric_transformer', numeric_transformer, train_X.select_dtypes(exclude=['object']).columns),
 ])

hgb_pipe = make_pipeline(Preprocessor, HistGradientBoostingClassifier())

search_space = {
    'histgradientboostingclassifier__learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3],
    'histgradientboostingclassifier__max_leaf_nodes': [None],
    'histgradientboostingclassifier__min_samples_leaf': [10, 20, 50, 100],
    'histgradientboostingclassifier__scoring': ['accuracy'],
    'histgradientboostingclassifier__loss': ['categorical_crossentropy'],
    'histgradientboostingclassifier__random_state': [53]
}


In [ ]:

#instantiate the Random CV Search
hgb_grid = RandomizedSearchCV(hgb_pipe, search_space, 
                              n_jobs=-1, 
                              cv=5, 
                              scoring='accuracy',
                              verbose=10, 
                              refit=True)

hgb_grid.fit(train_X, train_Y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
